### 아이디어
- R이 뜬 경우를 True, B를 그 반대로 정의하면 2-SAT문제가 보인다.
- 3개의 조건문을 2-SAT으로 변화시키는 아이디어가 중요할 것 같다.
  - 3개에서 2개 이상을 만족한다는 절을 2개의 리터럴로 변환한다면 $(a \lor b) \land (a \lor c) \land (b \lor c)$로 생각할 수 있다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def tarjan(G):
  SCC, S, P = [], [], []
  D = [0] * len(G)
 
  s = [*range(len(G))]
  while s:
    x = s.pop()
    if x < 0:
      d = D[~x] - 1
      if P[-1] > d:
        SCC.append(S[d:])
        del S[d:], P[-1]
        for x in SCC[-1]:
          D[x] = -1
    elif D[x] > 0:
      while P[-1] > D[x]:
        P.pop()
    elif D[x] == 0:
      S.append(x)
      P.append(len(S))
      D[x] = len(S)
      s.append(~x)
      s += G[x]
  return SCC[::-1]
 
class TwoSat:
  def __init__(self, n):
    self.n = n
    self.graph = [[] for _ in range(2 * n)]
 
  def _imply(self, x, y):
    self.graph[x].append(y if y >= 0 else 2 * self.n + y)
 
  def either(self, x, y):
    """either x or y must be True"""
    self._imply(~x, y)
    self._imply(~y, x)
  
  def set(self, x):
    """x must be True"""
    self._imply(~x, x)

  def solve(self):
    SCC = tarjan(self.graph)
    order = [0] * (2 * self.n)
    for i, comp in enumerate(SCC):
      for x in comp:
        order[x] = i
    for i, v in enumerate(order[:self.n]):
      if v == order[~i]:
        return False, None
    return True, ["R" if v > order[~i] else "B" for i, v in enumerate(order[:self.n])]

def sol() :
  k, n = map(int, input().split())
  ts = TwoSat(k)
  f = lambda x, y : int(x)-1 if y == b"R" else -int(x)
  for _ in range(n):
    x1, y1, x2, y2, x3, y3 = input().split()
    F1, F2, F3 = f(x1, y1), f(x2, y2), f(x3, y3)
    ts.either(F1, F2) #적어도 2개 이상 참이여야 보상을 받을 수 있다.
    ts.either(F2, F3)
    ts.either(F3, F1)
  ok, ans = ts.solve()
  if not ok:
    sys.stdout.write("-1")
  else :
    sys.stdout.write("".join(map(str, ans)))

sol()

- 문자열이 들어가는데 io.BytesIO 썻다가 바로 입구컷 한번 당했다.
- inverse(x)와 x 정점의 정의 주의해서 정해야 한다. 템플릿은 0-index 기준이고, 문제는 1-index 기준이기 때문.
  - `f = lambda x, y : int(x)-1 if y == b"R" else -int(x)`